# 所有sampler都是承 torch.utils.data.sampler.Sampler 这个类
## 1、顺序采样 SequentialSampler
作用 ：接收一个 Dataset 对象，输出数据包中样本量的顺序索引。 
### 1）内部代码 

In [ ]:
class SequentialSampler(Sampler):
    def __init__(self, data_source):
        self.data_source = data_source
    def __iter__(self):
        return iter(range(len(self.data_source)))
    def __len__(self):
        return len(self.data_source)

# __init__  接收参数：Dataset 对象
# __iter__ 返回一个可迭代对象（返回的是索引值），因为 SequentialSampler 是顺序采样，所以返回的索引是顺序数值序列
# __len__  返回 dataset 中数据个数

### 2）使用举例

In [ ]:
import torch.utils.data.sampler as sampler

data = list([17, 22, 3, 41, 8])
seq_sampler = sampler.SequentialSampler(data_source=data)

for index in seq_sampler:
    print("index: {}, data: {}".format(index, data[index]))

## 2、随机采样 RandomSampler
作用 ：接收一个 Dataset 对象，输出数据包中样本量的随机索引 （可指定是否可重复）。
### 1）内部代码 

In [ ]:
class RandomSampler(Sampler):
    def __init__(self, data_source, replacement=False, num_samples=None):
        self.data_source = data_source
        self.replacement = replacement
        self._num_samples = num_samples

    def num_samples(self):
        if self._num_samples is None:
            return len(self.data_source)
        return self._num_samples

    def __len__(self):
        return self.num_samples

    def __iter__(self):
        n = len(self.data_source)
        if self.replacement:
            # 生成的随机数是可能重复的
            return iter(torch.randint(high=n, size=(self.num_samples,), dtype=torch.int64).tolist())
        # 生成的随机数是不重复的
        return iter(torch.randperm(n).tolist())

# 查看 torch.randperm() 的使用 ：
# __init__ 参数 ：
#   data_source (Dataset): 采样的 Dataset 对象
#   replacement (bool): 如果为 True，则抽取的样本是有放回的。默认为 False
#   num_samples (int):  抽取样本的数量，默认是len(dataset)。当 replacement 是 True 时，应被实例化
# __iter__ 返回一个可迭代对象（返回的是索引），因为 RandomSampler 是随机采样，所以返回的索引是随机的数值序列 （当 replacement=False 时，生成的排列是无重复的）
# __len__  返回 dataset 中样本量

### 2）使用举例

In [ ]:
import torch.utils.data.sampler as sampler

data = list([17, 22, 3, 41, 8])
seq_sampler = sampler.RandomSampler(data_source=data)

for index in seq_sampler:
    print("index: {}, data: {}".format(index, data[index]))

## 3、批采样 BatchSampler
作用 ：包装另一个采样器以生成一个小批量索引
### 1）内部代码 

In [ ]:
class BatchSampler(Sampler):
    def __init__(self, sampler, batch_size, drop_last):、
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last

    def __iter__(self):
        batch = []
        for idx in self.sampler:
            batch.append(idx)
            # 如果采样个数和batch_size相等则本次采样完成
            if len(batch) == self.batch_size:
                yield batch
                batch = []
        # for 结束后在不需要剔除不足batch_size的采样个数时返回当前batch
        if len(batch) > 0 and not self.drop_last:
            yield batch

    def __len__(self):
        # 在不进行剔除时，数据的长度就是采样器索引的长度
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else:
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size

# 参数 ：
# sampler : 其他采样器实例
# batch_size ：批量大小
# drop_last ：为 “True”时，如果最后一个batch 采样得到的数据个数小于batch_size，则抛弃最后一个batch的数据

### 2）使用举例

In [ ]:
import torch.utils.data.sampler as sampler
data = list([17, 22, 3, 41, 8])

seq_sampler = sampler.SequentialSampler(data_source=data)
batch_sampler = sampler.BatchSampler(seq_sampler, 2, False )

for index in batch_sampler:
    print(index)

# 重要说明 
如果你在 DataLoader(dataset, batch_sampler=batch_sampler) 中指定了参数 batch_sampler， 那么就不能再指定参数 batch_size、shuffle、sampler、和 drop_last 了，他们互斥。 

因为：

你在生成torch.utils.data.sampler.BatchSampler() 的时候，就已经制定过  batch_size、sampler、和 drop_last 这些参数了，

batch_sampler 与 shuffle 作用一致，所以也互斥

比如，如下代码就会报错，因为在 DataLoader 中重复指定了 batch_size

In [ ]:
random_sampler = sampler.RandomSampler(data_source=dataset)
batch_sampler = sampler.BatchSampler(random_sampler, batch_size=2, drop_last=False)
dataloader1 = DataLoader(dataset, batch_size=2, batch_sampler=batch_sampler)